#### Part 1 : Data Fetch: Import Dependencies, Read CSV and explore Data features  

- First section is set out to import all required dependencies and to read raw_csv file into dataframe 

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import average_precision_score
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# READ IN CSV as raw_csv
raw_csv = pd.read_csv("Resources/PS_20174392719_1491204439457_log.csv")
raw_csv.head()

##### Exploring existing data set:
The following section explores the raw data `raw_csv` to identify cleaning required .
- In this section we review the existing data set.

In [ ]:
# Explore csv info
raw_csv.info()

In [ ]:
## Show Summary of isFraud Type 
type_summary = pd.crosstab(raw_csv['type'], raw_csv['isFraud'])
type_summary

In [ ]:
# Explore unique variables
unique_counts=raw_csv.nunique()
print(unique_counts)

There are a total of 6,362,620 rows and 8,213 instances of Fraud which is 0.129%.  
We have identified through column 'type' that Fraud only occured in 2 transaction types : 'CASH_OUT' and 'TRANSFER'.  
All other Types should be dropped from the model as they do not appear to have fraud in this case. 

There are 743 Steps as hours.

In [ ]:
# Explore  Distribution of numerical data.
distribution_summary  = raw_csv.describe()
print(distribution_summary )

In [ ]:
# Explore Distribution of Categorical Data
# Generate the distribution summary for categorical columns
categorical_summary = {}
for col in raw_csv.select_dtypes(include=['object']):
    categorical_summary[col] = raw_csv[col].value_counts()

# Display the distribution summary for categorical columns
for col, counts in categorical_summary.items():
    print(f"\nDistribution for column '{col}':")
    print(counts)


In [ ]:
# Explore if Null exists
raw_csv.isnull().values.any()

In [ ]:
# Count the occurrences of 0 in 'oldbalanceORG'
missing_oldbalance = (raw_csv['oldbalanceOrg'] == 0).sum()

# Print the result
print(missing_oldbalance)

In [ ]:
# Count the occurrences of 0 in 'amount'
missing_transaction = (raw_csv['amount'] == 0).sum()

# Print the result
print(missing_transaction)

#### Part 2: Data Clean and Create New Features & Digitise data

In [ ]:
# Copy raw_csv and use clean_csv to clean up and add features
clean_csv_df = raw_csv.copy()

In [ ]:

# Drop rows where the 'type' is 'Cash_IN', 'Debit', or 'Payment' in-place
types_to_drop = ['CASH_IN', 'DEBIT', 'PAYMENT']
clean_csv_df.drop(clean_csv_df[clean_csv_df['type'].isin(types_to_drop)].index, inplace=True)
clean_csv_df.head()

In [ ]:
# Drop rows where 'amount' is equal to 0
clean_csv_df = clean_csv_df[clean_csv_df['amount'] != 0]

In [ ]:
# Replace 0 values in 'oldbalanceORG' column with values from 'amount' column
clean_csv_df.loc[clean_csv_df['oldbalanceOrg'] == 0, 'oldbalanceOrg'] = clean_csv_df.loc[clean_csv_df['oldbalanceOrg'] == 0, 'amount']

In [ ]:
### Reset index and view data infom
clean_csv_df.reset_index(drop=True, inplace=True)
clean_csv_df.info()

In [ ]:
### NEW FEATURES based on Step : Hour  Cycle and Week

## Create hour cycle to reset to 1 every 24 steps
clean_csv_df['hour'] = ((clean_csv_df['step'] - 1) % 24) + 1

## Create Cycle every 24 Hours
# Create a new column 'new_column' initialized with 1
clean_csv_df['day'] = 1

# Calculate the increments of 1 for every 24 increases in the 'step' column
clean_csv_df['day'] += (clean_csv_df['step'] - 1) // 24


# Create a new column 'week' by increasing 1 for each cycle
clean_csv_df['week'] = ((clean_csv_df['step'] - 1) // 168) + 1

# Drop the intermediate 'cycles' column if needed
# raw_csv.drop(columns=['cycles'], inplace=True)

clean_csv_df.head()

In [ ]:
### NEW FEATURES based on old and new balance

# Create new columns for the division of 'amount' by 'oldbalanceOrg'
clean_csv_df['trans_weight'] = clean_csv_df['amount'] / clean_csv_df['oldbalanceOrg']

# % Change in Balance Transaction 
clean_csv_df['bal_change_per'] = (clean_csv_df['newbalanceOrig']-   clean_csv_df['oldbalanceOrg'])/ clean_csv_df['oldbalanceOrg']


# Create Large Transaction Figure
clean_csv_df['large_transaction'] = (clean_csv_df['amount'] > 100000).astype(int)


# Round 'trans_weight' to the nearest two decimals
clean_csv_df['trans_weight'] = clean_csv_df['trans_weight'].round(2)

# Round 'bal_change_per' to the nearest two decimals
clean_csv_df['bal_change_per'] = clean_csv_df['bal_change_per'].round(2)


clean_csv_df.head()



In [ ]:
### NEW FEATURES Creating Dummies bases on 'type'
# Create dummy variables for the 'type' column
dummy_columns = pd.get_dummies(clean_csv_df['type'], prefix='type')

# Concatenate the dummy variables with the original DataFrame
clean_csv_df = pd.concat([clean_csv_df, dummy_columns], axis=1)

# Display the updated DataFrame with the dummy columns
clean_csv_df.head()


In [ ]:
#######################
############# fraud_det_df
#########################
## PRINT the cleaned data to csv fraud_det_df
fraud_det_df = clean_csv_df.copy()

# Export the DataFrame as a CSV file. 
# fraud_det_df.to_csv("Resources/fraud_det_df.csv", encoding='utf8', index=False)
fraud_det_df.head()


In [ ]:
# Copy and confirm column features
fraud_det_df.columns

In [ ]:
#####################
## Copy dataframe to convert data for digital transformation
fraud_det_dig_df = fraud_det_df.copy()
fraud_det_dig_df.head()

In [ ]:
# Drop 'nameOrig' and 'type' columns
fraud_det_dig_df.drop(columns=['nameOrig', 'type'], inplace=True)

In [ ]:

# Digitize 'nameDest' based on categories and create the 'Destination' column
fraud_det_dig_df['Destination'] = pd.factorize(fraud_det_dig_df['nameDest'])[0]

# Print the updated DataFrame to check the result
fraud_det_dig_df.head()

In [ ]:
# Drop nameDest as has been factorised
fraud_det_dig_df.drop(columns=['nameDest'], inplace=True)


In [ ]:
#######################
############# fraud_det_dig_df
#########################
# View summary of digitised dataframe for model
fraud_det_dig_df.head()

#### Part 3 : Data Summary and Visualisation

In [ ]:
## Validate that no Null values have carried through.
null_values_count = fraud_det_df.isnull().sum()
null_values_count

In [ ]:
## Show Summary of isFraud Type 
type_summary = pd.crosstab(fraud_det_df['type'], fraud_det_df['isFraud'])
type_summary

In [ ]:
## Show Summary of isFraud Type 
type_summary = pd.crosstab(fraud_det_df['type'], fraud_det_df['isFlaggedFraud'])
type_summary

In [ ]:
## Show Summary of isFraud hour 
hour_summary = pd.crosstab(fraud_det_df['hour'], fraud_det_df['isFraud'])
hour_summary

In [ ]:
## Show Summary of isFraud hour 
week_summary = pd.crosstab(fraud_det_df['week'], fraud_det_df['isFraud'])
week_summary

In [ ]:
# Plot the distribution of 'fraud_status' with labels and y-axis in millions
# Plot the distribution of 'fraud_status' with labels and y-axis in millions
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
ax = sns.countplot(data=fraud_det_df, x='isFraud')

# Add labels to the bars
for p in ax.patches:
    ax.annotate(f"{p.get_height():,}", (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', fontsize=12, color='black', xytext=(0, 5), textcoords='offset points')

plt.title("Distribution of Fraudulent Transactions")
plt.xlabel("Fraud Status")
plt.ylabel("Count (Millions)")
plt.show()

##
count_fraud = fraud_det_df['isFraud'].value_counts()
print(count_fraud)

Exploratory Data Analysis and EDA and Fraud detection

The provided data has the financial transaction data as well as the target variable isFraud, which is the actual fraud status of the transaction and isFlaggedFraud is the indicator which the simulation is used to flag the transaction using some threshold.

The goal should be how we can improve and come up with better threshold to capture the fraud transaction.

In this visualization, we took a smaller random sample of 1000 transactions from the original dataset to make it easier to visualize. Then, we created three side-by-side histograms, one for each numeric feature ('amount', 'oldbalanceOrg', 'newbalanceOrig'), showing the distribution of these values to help understand how the data is spread out.

In [ ]:
# These are the numerical features in your dataset
numeric_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig']
sampled_data = fraud_det_df.sample(n=1000)  

plt.figure(figsize=(12, 6))
for i, feature in enumerate(numeric_features, 1):
    plt.subplot(1, len(numeric_features), i)
    sns.histplot(data=sampled_data, x=feature, kde=True)
    plt.title(f"Distribution of {feature}")
    plt.xlabel(feature)
    plt.ylabel("Frequency")
plt.tight_layout()
plt.show()



In [ ]:
# Bivariate Analysis of Categorical Features and 'isFraud'
categorical_features = ['type', 'hour', 'day', 'week']

for feature in categorical_features:
    plt.figure(figsize=(10, 6))
    sns.countplot(data=fraud_det_df, x=feature, hue='isFraud')
    plt.title(f"Fraud Distribution by {feature}")
    plt.xlabel(feature)
    plt.ylabel("Count")
    plt.legend(title='Fraud Status', labels=['Not Fraud', 'Fraud'])
    plt.show()



The above visualizations helps us compare the occurrence of fraud between different categories of categorical features. By visually examining the distribution of fraudulent and non-fraudulent transactions, we can identify patterns and trends related to fraud occurrence in specific scenarios, such as transaction types, hours, days, or weeks. These insights enable fraud investigators to focus their efforts on high-risk categories and implement targeted fraud prevention strategies.

In [ ]:
# Bivariate Analysis of Numeric Features and 'isFraud'
numeric_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig']

for feature in numeric_features:
    plt.figure(figsize=(8, 6))
    sns.boxplot(data=fraud_det_df, x='isFraud', y=feature)
    plt.title(f"{feature} vs. Fraud Status")
    plt.xlabel("Fraud Status")
    plt.ylabel(feature)
    plt.show()


We use bivariate analysis using box plots provides valuable insights into the distribution of numeric features based on fraud status. This information helps in designing more effective fraud detection strategies and improving the accuracy of detecting fraudulent transactions.

In [ ]:
# # Outlier detection 
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Numeric features for outlier detection
# numeric_features = ['amount', 'oldbalanceOrg', 'newbalanceOrig']

# # Create box plots for numeric features
# plt.figure(figsize=(12, 6))
# for i, feature in enumerate(numeric_features, 1):
#     plt.subplot(1, len(numeric_features), i)
#     sns.boxplot(data=fraud_det_df, x=feature)
#     plt.title(f"Box Plot of {feature}")
#     plt.xlabel(feature)
# plt.tight_layout()
# plt.show()


#### Part 4: Print CSV for Data Model

In [ ]:
## Print fraud_det_df which is first working data frame
### Data is not digitised and still contains dummies

# fraud_det_df.to_csv("Resources/fraud_det.csv", index=False)

In [ ]:
## Print fraud_det__dig_df which is first working data frame
# ### Data is not digitised and still contains dummies
# fraud_det_dig_df.to_csv("Resources/fraud_det_dig_df.csv", index=False)

In [ ]:
fraud_det_df.head()


In [ ]:
fraud_det_dig_df.head()

#### Part 5 : Addresing Class Imbalance

In [ ]:
raw_df = pd.read_csv("Resources/fraud_det_dig_df.csv")


In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

# Visualize the distribution of 'isFraud' before balancing
plt.figure(figsize=(8, 6))
ax = sns.countplot(data=raw_df, x='isFraud')

for p in ax.patches:
    ax.annotate(f"{p.get_height():,}", (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', fontsize=12, color='black', xytext=(0, 5), textcoords='offset points')

plt.title("Distribution of Fraudulent Transactions (Before Balancing)")
plt.xlabel("isFraud")
plt.ylabel("Count (Millions)")
plt.show()

## DEFINE target Class and set X variables
# Separate features and target variable
X = raw_df.drop('isFraud', axis=1)
y = raw_df['isFraud']

# Perform train-test split on the original DataFrame
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

######
## Variables to use for model
######
# Perform Random Undersampling
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X_train, y_train)
##X_rus_test, y_rus_test = rus.fit_resample(X_test,y_test)

######
##
######


# Visualize the distribution of 'isFraud' after Random Undersampling
plt.figure(figsize=(8, 6))
ax = sns.countplot(x=y_rus)

for p in ax.patches:
    ax.annotate(f"{p.get_height():,}", (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', fontsize=12, color='black', xytext=(0, 5), textcoords='offset points')

plt.title("Distribution of Fraudulent Transactions (After Random Undersampling)")
plt.xlabel("isFraud")
plt.ylabel("Count")
plt.show()

# Perform Random Oversampling
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X_train, y_train)

# Visualize the distribution of 'isFraud' after Random Oversampling
plt.figure(figsize=(8, 6))
ax = sns.countplot(x=y_ros)

for p in ax.patches:
    ax.annotate(f"{p.get_height():,}", (p.get_x() + p.get_width() / 2., p.get_height()), 
                ha='center', va='center', fontsize=12, color='black', xytext=(0, 5), textcoords='offset points')

plt.title("Distribution of Fraudulent Transactions (After Random Oversampling)")
plt.xlabel("isFraud")
plt.ylabel("Count")
plt.show()


### Data Fetch for Tableau Visualisation
Fetch Data for TABLEAU Visualisation - minimise and aggregate where possible

In [ ]:
tableau_df = pd.read_csv("Resources/PS_20174392719_1491204439457_log.csv")
tableau_df.columns

In [ ]:
# Recreate time and hour features
tableau_df['hour'] = ((tableau_df['step'] - 1) % 24) + 1

## Create Cycle every 24 Hours
# Create a new column 'new_column' initialized with 1
tableau_df['day'] = 1

# Calculate the increments of 1 for every 24 increases in the 'step' column
tableau_df['day'] += (tableau_df['step'] - 1) // 24

# Create a new column 'week' by increasing 1 for each cycle
tableau_df['week'] = ((tableau_df['step'] - 1) // 168) + 1

tableau_df['time'] = (tableau_df['hour'] - 1).apply(lambda x: f"{x % 12 + 1}:00 {'AM' if x < 12 else 'PM'}")
tableau_df.head()


In [ ]:
## Dtop columns not to use for summary
columns_to_drop = ['step', 'hour', 'isFlaggedFraud']
tableau_df.drop(columns_to_drop, axis=1, inplace=True)
tableau_df.head()

In [ ]:
# AGGFREGATE DATA SUM
# Grouping by the specified columns and calculating the sum of 'amount' and 'oldbalanceOrg'
agg_sum_df = tableau_df.groupby(['type', 'isFraud', 'time', 'day', 'week']).agg({
    'amount': 'sum',
    'oldbalanceOrg': 'sum',
    'nameOrig': 'count'  # Calculates the count of 'nameOrig'
}).reset_index()

agg_sum_df.head()


agg_sum_df.to_csv("Resources/tableau_agg_sum.csv", index=False)


In [ ]:
## Create specialised Dataframe summary  

# Grouping by 'type' and 'isFraud' and aggregating the required statistics
agg_unique_df = tableau_df.groupby(['type', 'isFraud']).agg({
    'amount': ['mean', 'max', 'min'],
    'nameDest': pd.Series.nunique,
    'nameOrig': pd.Series.nunique  # Calculates the count of unique 'nameDest'
}).reset_index()

# Rename the columns for clarity
agg_unique_df.columns = ['type', 'isFraud', 'average_amount', 'max_amount', 'min_amount', 'unique_recipient','unique_customer']
agg_unique_df.to_csv("Resources/tableau_agg_unique.csv", index=False)
